In [3]:
import mlflow
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import imutils
from mlflow import MlflowClient

In [4]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\Clsm\Documents\GitHub\prompts-generate-ai\service_acc\sktblog-GOOGLE_APPLICATION_CREDENTIALS.json'

In [5]:
mlflow.set_tracking_uri("http://34.173.115.10:8888")
mlflow.set_experiment("epm-fruits-classification")

<Experiment: artifact_location='gs://aiteam-mlflow-storage-bucket/models/19', creation_time=1678935351328, experiment_id='19', last_update_time=1678935351328, lifecycle_stage='active', name='epm-fruits-classification', tags={}>

In [6]:
train_dir = r'C:\Users\Clsm\fruit_dataset\train'
test_dir = r'C:\Users\Clsm\fruit_dataset\test'

In [7]:
train_datagen = ImageDataGenerator(rescale= 1./255)
test_datagen = ImageDataGenerator(rescale= 1./255)

In [8]:
batch_size = 16
train_set = train_datagen.flow_from_directory(train_dir,
                                            target_size= (100,100),
                                            batch_size = batch_size,
                                            class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(test_dir,
                                            target_size = (100,100),
                                            batch_size = batch_size,
                                            class_mode = 'categorical')

Found 2301 images belonging to 10 classes.
Found 1025 images belonging to 10 classes.


In [9]:
y_test = test_set.classes
y_test

array([0, 0, 0, ..., 9, 9, 9])

In [10]:
IMAGE_SIZE = [100,100]
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#do not train the pre-trained layers of VGG-19
for layer in vgg.layers:
    layer.trainable = False
x = Flatten()(vgg.output)
#adding output layer.Softmax classifier is used as it is multi-class classification
prediction = Dense(10, activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=prediction)

In [11]:
model.compile(
  loss='categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [12]:
from sklearn.linear_model import Lasso
from sklearn.metrics import accuracy_score
import pickle
mlflow.sklearn.autolog()
with mlflow.start_run() as run:
    mlflow.set_tag("developer", "epm")
    mlflow.set_tag("version", "prerelease")

    mlflow.log_param('dataset' , 'fruit_classification')
    mlflow.log_param('train_amount', '2301images')
    mlflow.log_param('test_amount', '1025images')
    hist = model.fit(train_set, batch_size = batch_size, epochs= 1)
    mlflow.sklearn.log_model(hist, artifact_path="fruits-model_test")

name = "fruit_classification_test"
client = MlflowClient()
client.create_registered_model(name)

2023/03/16 10:58:42 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


144/144 [==============================] - 242s 2s/step - loss: 1.5905 - accuracy: 0.4650
INFO:tensorflow:Assets written to: ram://12d2cb02-4cf2-4e2a-b8cb-f5aac978aad0/assets


2023/03/16 11:02:53 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023/03/16 11:03:05 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Clsm\AppData\Local\Temp\tmp_af7j9lw\model\model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.2.0', 'cloudpickle==2.2.0']. Set logging level to DEBUG to see the full traceback.


Forbidden: 403 POST https://storage.googleapis.com/upload/storage/v1/b/aiteam-mlflow-storage-bucket/o?uploadType=multipart: {
  "error": {
    "code": 403,
    "message": "sktblog@sktblog.iam.gserviceaccount.com does not have storage.objects.create access to the Google Cloud Storage object. Permission 'storage.objects.create' denied on resource (or it may not exist).",
    "errors": [
      {
        "message": "sktblog@sktblog.iam.gserviceaccount.com does not have storage.objects.create access to the Google Cloud Storage object. Permission 'storage.objects.create' denied on resource (or it may not exist).",
        "domain": "global",
        "reason": "forbidden"
      }
    ]
  }
}
: ('Request failed with status code', 403, 'Expected one of', <HTTPStatus.OK: 200>)

In [ ]:
accuracy = model.evaluate(test_set)
print(accuracy)

In [ ]:
y_pred = model.predict(test_set)
pred_values = []
for pred in y_pred:
    pred = np.argmax(pred)
    pred_values.append(pred)

In [ ]:
y_true = test_set.classes.tolist()
test_accuracy = accuracy_score(y_true, y_pred)
print(test_accuracy)